Для первой задачи мы используем данные [Jester Online Joke Recommender System](https://goldberg.berkeley.edu/jester-data/)

**Описание данных**

Файл `train_joke_df.csv` содержит:
- UID - id пользователей
- JID - id шуток, которые 
- Ratin - рейтинг шутки, который проставил пользователь 


Рейтинг имеет значение от -10.00 до 10.00. Могут встречаться значения 99.00, но это обозначает Null (нет рейтинга от пользователя).

Метрика для оценки [RMSE](https://www.codecamp.ru/blog/how-to-interpret-rmse/)

In [1]:
#!pip install "scikit-surprise==1.1.3"

In [2]:
#!pip install "xlrd==2.0.1"

### Import

In [3]:
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
from surprise import dump
from surprise import SVD
np.random.seed(42)

### Загрузка и обработка данных

In [4]:
df = pd.read_csv('data/recsys-in-practice/train_joke_df.csv')

df.head(5)

,UID,JID,Rating
0,18029,6,-1.26
1,3298,64,-4.17
2,3366,58,0.92
3,12735,92,3.69
4,11365,38,-6.60


In [5]:
# сделаем сортировку и перепишем index
df = df.sort_values(by=['UID', 'JID'])
df = df.reset_index(drop=True)

In [6]:
# создадим на основе набора данных
# поднабор, который требуется для библиотеки Surprise

# указываем минимальный и максимальный рейтинги
reader = Reader(rating_scale=(-10, 10))

# передаём набор, указывая последовательность колонок: user (raw) ids, item (raw) ids, ratings
# для Surprise - это обязательно
data = Dataset.load_from_df(df[['UID', 'JID', 'Rating']], reader)

In [7]:
trainset_data = data.build_full_trainset()

# сделаем разделение на обучающую и тестовую выборку
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

### Обучение модели

In [8]:
# обучим с лучшими параметрами
algo = SVD(random_state=0, n_epochs=100, n_factors= 512)
algo.fit(trainset)

# получим предикт и посмотрим метрику
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 4.0689


4.0689312503214765

### Для отправки на тестирование

In [9]:
# обучим с лучшими параметрами на всех данных
algo = SVD(random_state=0, n_epochs=100, n_factors= 512)
algo.fit(trainset_data)

# получим предикт и посмотрим метрику
predictions = algo.test(trainset_data.build_testset())
accuracy.rmse(predictions)

RMSE: 0.2524


0.25238153948056485

In [10]:
test = pd.read_csv('data/recsys-in-practice/test_joke_df_nofactrating.csv', index_col=0)
test.head(5)

,UID,JID
InteractionID,,
0,11228,39
1,21724,85
2,16782,56
3,12105,42
4,14427,2


In [11]:
test['Rating'] = test[['UID', 'JID']].apply(lambda x: algo.predict(x[0], x[1], verbose=False).est,
                                                      axis = 1)

In [12]:
# вид набора данных, который должен быть отправлен для тестирования
test['Rating'].to_frame().head(5)

,Rating
InteractionID,
0,3.042851
1,-7.039993
2,-0.331728
3,6.796138
4,5.779301


In [13]:
# формирование файла для отправки в Kaggle
test['Rating'].to_frame().to_csv('baseline_svd.csv')

In [14]:
file_name = 'baseline_svd'
dump.dump(file_name, algo=algo)
#_, loaded_algo = dump.load(file_name)